In [1]:
# Importing Libraries
import selenium
import pandas as pd
import time
from bs4 import BeautifulSoup

# Importing selenium webdriver 
from selenium import webdriver

# Importing required Exceptions which needs to handled
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException

#Importing requests
import requests

# importing regex
import re

In [3]:
# Activating the chrome browser
driver = webdriver.Chrome("chromedriver.exe") 
time.sleep(1)

# Opening the homepage of carwale
url = "https://www.amazon.in/"
driver.get(url)

In [4]:
#list of the products 
products = ["smart Phone","laptop", "headphones","monitor","smart watch","professional camera","printer","router","home theater"]

columns = ["product type", "title", "review", "rating"]

In [5]:
#creating an empty dataframe to which the data scraped will be added
df2 = pd.DataFrame({} , columns = columns)

In [6]:
#making a loop
for p in products: 
    
    #fetching thesearch box
    search_bar = driver.find_element_by_id("twotabsearchtextbox")
    
    #fetching the search button
    search_btn = driver.find_element_by_id("nav-search-submit-button") 
    
    #clearing the previous text for loop
    search_bar.clear() 
    
    #fetching input from product list
    search_bar.send_keys(p)
    
    #clicking on the search button
    search_btn.click() 
    
    #scrolling to the page to the bottom to get all the data
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight);") 
    
    #scrapping the required details
    start=0
    end=2
    for page in range(start,end):#for loop for scrapping 3 pages
        search_list = driver.find_elements_by_xpath("//div[@class='a-section a-spacing-none']//h2//a//span")
        for i in search_list:
            product = i.text
            try:
                i.click()
            except:
                continue
            
            #switching to the product page to get the rating
            driver.switch_to.window(driver.window_handles[-1])
            
            #scrolling down the page
            driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
            
            try:
                stars = driver.find_element_by_xpath("//table[@id = 'histogramTable']//tr//td[1]//span//a")
                stars.click() #getting the review page
            except:
                print("No reviews available " , product) #if reviews are not available
            
            #again scrolling down to get the ratings
            driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
            
            #getting the star ratings
            stars_select = driver.find_elements_by_xpath("//table[@id = 'histogramTable']//tr//td[1]//span//a")
            time.sleep(2)
            
            #making the loop for getting stars rating
            for s in stars_select: 
                s.click() 
                time.sleep(1)
                
                #getting the 3 pages to iterate the information
                for num in range(0,3):
                    
                    #fetching the review title
                    reviews_title = driver.find_elements_by_xpath("//div[@data-hook = 'review']//a[@data-hook = 'review-title']")
                    
                    #scraping the individual row
                    for j in range(len(reviews_title)): 
                        
                        row = []
                        
                        try: 
                            #getting the review data
                            reviews_title = driver.find_element_by_xpath("//div[@data-hook = 'review'][{}]//a[@data-hook = 'review-title']".format(j+1))
                            reviews_content =  driver.find_element_by_xpath("//div[@data-hook = 'review'][{}]//span[@data-hook = 'review-body']//span".format(j+1))
                            
                            
                            #getting the text data only
                            row.append(p)
                            row.append(reviews_title.text)
                            row.append(reviews_content.text)
                            row.append(s.text)
                            
                        except:
                            continue #skips the loop if data not available
                            
                        df1 = pd.DataFrame([row] , columns=columns)
                        df = pd.concat([df,df1],ignore_index=True)
                
                    try:
                        next_btn  = driver.find_element_by_xpath("//div[@role='navigation']//li[@class='a-last']//a")
                        next_btn.click()
                        time.sleep(2)
                        
                    except:
                        break #breaks the loop if there are no more reviews.
        
            driver.close() #close the reviews page
            driver.switch_to_window(driver.window_handles[0]) #switching to the start page
            
        #scrolling down    
        driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
        
        #next page of search_list
        next_btn  = driver.find_element_by_xpath("//div[@role='navigation']//ul[@class='a-pagination']//li[@class='a-last']//a")
        next_btn.click()
        time.sleep(2)

<ipython-input-6-18f140ea4674>:78: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[0]) #switching to the main page (search list page)


No reviews for the product  I KALL K500 Smartphone (4GB, 64GB) (Red)
No reviews for the product  I KALL K8 Smartphone (2GB, 16GB) (Green)
No reviews for the product  I KALL K280 Smartphone (6.26 Inch, 3GB, 32GB, Dual Sim, 4G Volte) (Grey)
No reviews for the product  I KALL K280 Smartphone (6.26 Inch, 3GB, 32GB, Dual Sim, 4G Volte) (Black)
No reviews for the product  I KALL K200 Smartphone (2GB, 16GB) (Blue)
No reviews for the product  I KALL K8 Smartphone (2GB, 16GB) (Red)
No reviews for the product  Lenovo IdeaPad Slim 3 10th Gen Intel Core i5 15.6" (39.62cm) FHD IPS Thin & Light Laptop (8GB/512GB SSD/Windows 11/MS Office 2021/2 Yr Warranty/Platinum Grey/1.7Kg), 81WB01BJIN
No reviews for the product  Lenovo Ideapad Gaming 3 AMD Ryzen 7 5800H 15.6" (39.62cm) FHD IPS Gaming Laptop (16GB/512GB SSD/Windows 11/Nvidia GTX 1650 4GB/120Hz Refresh Rate/Backlit Keyboard/Shadow Black/2.25Kg), 82K200X2IN
No reviews for the product  HP 15s 11th Gen Intel Core i3 15.6 inches(39.6 cm) Laptop ( 8GB R

In [7]:
df2

,product type,title,review,rating
0,smart Phone,It is beyond Expectations♥️,1. Specifications wise very good phone.\n2.Cam...,5 star
1,smart Phone,Perfect phone in mid range segment,I am not a techie guy and thus giving my revie...,5 star
2,smart Phone,Waste of money worst phone in my life,Great phone for pubg lovers I played call of d...,5 star
3,smart Phone,Just loved it 😃,Awesome phone\nOnly camera optimization needed...,5 star
4,smart Phone,After 2 weeks of usage Lcd IPS display bleedin...,Due to faulty lcd display\nGot full phone rep...,5 star
...,...,...,...,...
35983,home theater,"Not good ,don't bay","Not up to mark ,sound quality not good , looki...",1 star
35984,home theater,Speakers distort and rattle within a month of ...,The speaker quality is very bad. It gives out ...,1 star
35985,home theater,Works within warranty only. Immediately fail.,"Company warranty three months , Fourth month g...",1 star
35986,home theater,😟😟😟,This iteam was very bad quality and worst prod...,1 star


In [8]:
df2.to_excel('df2.xlsx')